In [2]:
! pwd

/Users/Home/Documents/QML/Hackathon/qml_hackathon


In [2]:
import os

import pandas as pd
import numpy as np

X_tr = np.load('X_train.npy')
X_te = np.load('X_test.npy')
y1_tr = np.load('y1_train.npy')
y2_tr = np.load('y2_train.npy')

# Fill in your API token:


sapi_token = 'CDL8-df1de1d5d76560ee73a82ffca3833a1a444536d3'
url = 'https://cloud.dwavesys.com/sapi'
token = sapi_token
solver_name = 'c4-sw_sample'#'DW_2000Q_2'

# import necessary packages
from sklearn import preprocessing, metrics
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.datasets.mldata import fetch_mldata
from sklearn.datasets import load_breast_cancer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

from qboost import WeakClassifiers, QBoostClassifier, QboostPlus


# Define the functions required in this example

def metric(y, y_pred):
    """
    :param y: true label
    :param y_pred: predicted label
    :return: metric score
    """

    return metrics.accuracy_score(y, y_pred)

# performance metric

def train_model(X_train, y_train, X_test, y_test, lmd):
    """
    :param X_train: training data
    :param y_train: training label
    :param X_test: testing data
    :param y_test: testing label
    :param lmd: lambda used in regularization
    :return:
    """

    # define parameters used in this function
    NUM_READS = 1000
    NUM_WEAK_CLASSIFIERS = 30
    TREE_DEPTH = 4
    DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 'num_spin_reversal_transforms': 10,
                 'postprocess': 'optimization',
                 }

    # define sampler
    dwave_sampler = DWaveSampler(token=sapi_token, endpoint = url)
    emb_sampler = EmbeddingComposite(dwave_sampler)

    N_train = len(X_train)
    N_test = len(X_test)
    print("\n======================================")
    print("Train size: %d, Test size: %d" %(N_train, N_test))
    print('Num weak classifiers:', NUM_WEAK_CLASSIFIERS)

    # Preprocessing data
    imputer = preprocessing.Imputer()
    scaler = preprocessing.StandardScaler()
    normalizer = preprocessing.Normalizer()

    X_train = scaler.fit_transform(X_train)
    X_train = normalizer.fit_transform(X_train)

    X_test = scaler.fit_transform(X_test)
    X_test = normalizer.fit_transform(X_test)
    
    ## Adaboost
    print('\nAdaboost')
    clf1 = AdaBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS)
    clf1.fit(X_train, y_train)
    y_train1 = clf1.predict(X_train)
    y_test1 = clf1.predict(X_test)
#     print(clf1.estimator_weights_)
    print('accu (train): %5.2f'%(metric(y_train, y_train1)))
    print('accu (test): %5.2f'%(metric(y_test, y_test1)))

        # Ensembles of Decision Tree
    print('\nDecision tree')
    clf2 = WeakClassifiers(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf2.fit(X_train, y_train)
    y_train2 = clf2.predict(X_train)
    y_test2 = clf2.predict(X_test)
#     print(clf2.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train2)))
    print('accu (test): %5.2f' % (metric(y_test, y_test2)))
    
    # Random forest
    print('\nRandom Forest')
    clf3 = RandomForestClassifier(max_depth=TREE_DEPTH, n_estimators=NUM_WEAK_CLASSIFIERS)
    clf3.fit(X_train, y_train)
    y_train3 = clf3.predict(X_train)
    y_test3 = clf3.predict(X_test)
    print('accu (train): %5.2f' % (metric(y_train, y_train3)))
    print('accu (test): %5.2f' % (metric(y_test, y_test3)))
    
    # Qboost
    print('\nQBoost')
    clf4 = QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf4.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train4 = clf4.predict(X_train)
    y_test4 = clf4.predict(X_test)
    print(clf4.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train4)))
    print('accu (test): %5.2f' % (metric(y_test, y_test4)))

#    QboostPlus
    print('\nQBoostPlus')
    clf5 = QboostPlus([clf1, clf2, clf3, clf4])
    clf5.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train5 = clf5.predict(X_train)
    y_test5 = clf5.predict(X_test)
    print(clf5.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train5)))
    print('accu (test): %5.2f' % (metric(y_test, y_test5)))

    
    return [clf4]

# start training the model

idx = np.arange(len(X_tr))
np.random.shuffle(idx)  # shuffles index

y_train = y1_tr
y_bin = 2*(y1_tr >0.25) - 1

X_train = X_tr[:int(len(idx)*.8)]
y_train = y_bin[:int(len(idx)*.8)]

X_test = X_tr[int(len(idx)*.8):]
y_test = y_bin[int(len(idx)*.8):]

# start training the model
#X_train = X_tr
#y_train = y1_tr
#y_train = 2*(y_train >0.25) - 1
#X_test = X_train
#y_test = y_train


In [3]:
clfs = train_model(X_train, y_train, X_test, y_test, 1.0)



Train size: 1920, Test size: 480
('Num weak classifiers:', 30)

Adaboost
accu (train):  0.93
accu (test):  0.89

Decision tree
accu (train):  0.97
accu (test):  0.91

Random Forest
accu (train):  0.92
accu (test):  0.90

QBoost
[1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1]
accu (train):  0.94
accu (test):  0.88

QBoostPlus
[1 1 1 1]
accu (train):  0.95
accu (test):  0.91


In [4]:
# create bins and allocate y to each bin

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

y = y1_tr
x = X_tr

# split into n_classes, or into n_classes bins
n_classes = 10
bins = np.linspace(np.min(y), np.max(y), num=n_classes)
inds = np.digitize(y, bins)

inds

array([2, 4, 3, ..., 2, 5, 2])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
from sklearn.metrics import accuracy_score
# implement one versus rest classifier

random_state = np.random.RandomState(0)
y = inds # label_binarize(inds, classes=bins)
#n_classes = y.shape[1]
print np.min(y),np.max(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2,
                                                    random_state=0)
print len(X_test)

#OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)

#classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
#y_score = classifier.fit(X_train, y_train)
#y_pred = classifier.predict(X_test)
#acc = accuracy_score(y_test, y_pred)
#print acc

1 10
480


In [7]:
# run qboost with dicrete variables
# does not work well

clfs = train_model(X_train, y_train, X_test, y_test, 1.0)


Train size: 1920, Test size: 480
('Num weak classifiers:', 30)

Adaboost
accu (train):  0.29
accu (test):  0.27

Decision tree
accu (train):  0.10
accu (test):  0.08

Random Forest
accu (train):  0.61
accu (test):  0.51

QBoost
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
accu (train):  0.10
accu (test):  0.08

QBoostPlus
[1 1 1 1]
accu (train):  0.10
accu (test):  0.08


In [9]:
# one vs rest classifer with AdaBoost
# works well, but low performance due to unbalances classes 
NUM_WEAK_CLASSIFIERS=30
classifier = OneVsRestClassifier(AdaBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS))
y_score = classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print y_pred, acc

[4 4 2 1 6 3 1 2 5 6 4 4 4 2 2 3 2 7 3 4 2 3 2 4 1 1 3 4 4 1 4 3 4 4 4 3 2
 6 2 5 1 2 2 4 1 5 1 3 2 4 1 6 2 4 4 2 1 2 3 2 2 2 2 3 6 4 3 4 4 3 6 3 6 2
 3 5 5 2 4 2 1 4 2 3 3 3 5 3 4 4 2 3 5 5 4 3 4 3 3 3 4 2 3 4 6 5 4 3 5 6 1
 4 3 3 2 1 4 5 5 2 4 2 4 5 6 5 4 2 3 6 3 5 2 4 5 2 7 4 4 2 6 3 1 6 2 2 2 2
 5 4 5 3 5 5 5 4 3 3 6 2 3 4 4 6 2 2 4 6 2 3 2 4 5 7 4 4 3 4 2 2 6 5 2 4 4
 5 5 4 3 5 4 4 3 3 3 2 2 2 1 3 1 2 3 3 3 6 3 4 5 4 3 4 2 3 5 5 4 3 3 4 3 2
 4 1 2 1 2 4 3 3 4 4 5 4 4 4 3 3 2 4 2 2 6 3 3 5 1 2 3 1 4 5 4 4 2 3 4 7 4
 2 6 4 2 3 6 2 6 4 2 4 2 5 3 4 4 2 7 2 2 4 4 4 6 3 4 4 4 4 5 3 6 2 5 2 3 5
 4 2 3 1 2 3 2 4 5 6 3 4 3 5 3 1 4 2 3 2 4 4 6 1 3 4 2 2 1 5 6 2 3 4 2 3 5
 3 6 3 5 3 4 2 2 2 2 5 5 3 3 2 4 4 2 3 3 4 6 4 3 4 3 2 2 4 4 4 3 2 2 3 4 4
 2 2 6 4 2 4 4 2 6 5 5 2 2 2 4 2 5 1 3 2 5 4 2 6 3 3 4 5 5 3 2 3 6 4 3 5 4
 3 4 2 2 4 3 3 3 4 5 3 6 3 4 1 4 3 4 3 6 1 4 2 6 2 4 3 3 3 5 5 2 4 1 3 5 6
 4 4 4 6 2 4 5 1 4 4 4 3 5 4 3 1 6 7 1 3 4 6 3 4 3 3 5 3 3 4 4 6 4 6 6 5] 0.6125


In [50]:
## One vs Rest classifier

y = y1_tr
x = X_tr

# split into n_classes, or into n_classes bins
n_classes = 3
bins = np.linspace(np.min(y), np.max(y), num=n_classes)
inds = np.digitize(y, bins)


# split data

X_train, X_test, y_train, y_test = train_test_split(x, inds, test_size=.2,
                                                    random_state=0)
##

NUM_READS = 1000
NUM_WEAK_CLASSIFIERS = 30
TREE_DEPTH = 4
DW_PARAMS = {'num_reads': NUM_READS,
             'auto_scale': True,
             'num_spin_reversal_transforms': 10,
             'postprocess': 'optimization',
             }

# define sampler
dwave_sampler = DWaveSampler(token=sapi_token, endpoint = url)
emb_sampler = EmbeddingComposite(dwave_sampler)
lmd = 0.2

classifiers = []
predictions = []
pred_test_labels = [0]*len(y_test)
pred_train_labels = [0]*len(y_train)

for i in range(n_classes):
    print 'Working on Qboost: ',i+1,'/',n_classes
    new_label = 2*(y_train==i+1)-1
     
    new_label = np.array(new_label)
    clf = QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf.fit(X_train, new_label, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    classifiers.append(clf)
    predictions.append(y_test_pred)

for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        if predictions[i][j] == 1:
            pred_test_labels[j] = i+1
            
print pred_test_labels
acc = accuracy_score(y_test, pred_test_labels)

print 'Accuracy Score:', acc

Working on Qboost:  1 / 3
Working on Qboost:  2 / 3
Working on Qboost:  3 / 3
[1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 

In [48]:
new_label

array([-1, -1, -1, ..., -1, -1, -1])

In [65]:
def qboost_single(X_train, y_train, X_test, y_test, lmd):
    """
    :param X_train: training data
    :param y_train: training label
    :param X_test: testing data
    :param y_test: testing label
    :param lmd: lambda used in regularization
    :return:
    """

    # define parameters used in this function
    NUM_READS = 1000
    NUM_WEAK_CLASSIFIERS = 30
    TREE_DEPTH = 4
    DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 'num_spin_reversal_transforms': 10,
                 'postprocess': 'optimization',
                 }

    # define sampler
    dwave_sampler = DWaveSampler(token=sapi_token, endpoint = url)
    emb_sampler = EmbeddingComposite(dwave_sampler)

    N_train = len(X_train)
    N_test = len(X_test)
    print("\n======================================")
    print("Train size: %d, Test size: %d" %(N_train, N_test))
    print('Num weak classifiers:', NUM_WEAK_CLASSIFIERS)

    # Preprocessing data
    imputer = preprocessing.Imputer()
    scaler = preprocessing.StandardScaler()
    normalizer = preprocessing.Normalizer()

    X_train = scaler.fit_transform(X_train)
    X_train = normalizer.fit_transform(X_train)

    X_test = scaler.fit_transform(X_test)
    X_test = normalizer.fit_transform(X_test)
    
    ## Adaboost
    print('\nAdaboost')
    clf1 = AdaBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS)
    clf1.fit(X_train, y_train)
    y_train1 = clf1.predict(X_train)
    y_test1 = clf1.predict(X_test)
#     print(clf1.estimator_weights_)
    print('accu (train): %5.2f'%(metric(y_train, y_train1)))
    print('accu (test): %5.2f'%(metric(y_test, y_test1)))

        # Ensembles of Decision Tree
    print('\nDecision tree')
    clf2 = WeakClassifiers(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf2.fit(X_train, y_train)
    y_train2 = clf2.predict(X_train)
    y_test2 = clf2.predict(X_test)
#     print(clf2.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train2)))
    print('accu (test): %5.2f' % (metric(y_test, y_test2)))
    
    # Random forest
    print('\nRandom Forest')
    clf3 = RandomForestClassifier(max_depth=TREE_DEPTH, n_estimators=NUM_WEAK_CLASSIFIERS)
    clf3.fit(X_train, y_train)
    y_train3 = clf3.predict(X_train)
    y_test3 = clf3.predict(X_test)
    print('accu (train): %5.2f' % (metric(y_train, y_train3)))
    print('accu (test): %5.2f' % (metric(y_test, y_test3)))
    
    # Qboost
    print('\nQBoost')
    clf4 = QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf4.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train4 = clf4.predict(X_train)
    y_test4 = clf4.predict(X_test)
    print(clf4.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train4)))
    print('accu (test): %5.2f' % (metric(y_test, y_test4)))

#    QboostPlus
    print('\nQBoostPlus')
    clf5 = QboostPlus([clf1, clf2, clf3, clf4])
    clf5.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train5 = clf5.predict(X_train)
    y_test5 = clf5.predict(X_test)
    print(clf5.estimator_weights)
    print('accu (train): %5.2f' % (metric(y_train, y_train5)))
    print('accu (test): %5.2f' % (metric(y_test, y_test5)))
    
#    Q_OneVsRest Plus
    print('\nQ_OneVsRest')
    clf6 = OneVsRestClassifier(QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH))
    clf6.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train6 = clf6.predict(X_train)
    y_test6 = clf6.predict(X_test)
    print(clf6.estimator_weights)

    
    return [clf4]

In [66]:
clfs = train_model(X_train, y_train, X_test, y_test, 1.0)




Train size: 1920, Test size: 480
('Num weak classifiers:', 30)

Adaboost
accu (train):  0.29
accu (test):  0.27

Decision tree
accu (train):  0.10
accu (test):  0.08

Random Forest
accu (train):  0.62
accu (test):  0.54

QBoost
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
accu (train):  0.10
accu (test):  0.08

QBoostPlus
[1 1 1 1]
accu (train):  0.10
accu (test):  0.08


In [81]:
import array
import numpy as np
import warnings
import scipy.sparse as sp
import itertools

import BaseEstimator, ClassifierMixin, clone, is_classifier
from .base import MetaEstimatorMixin, is_regressor
from .preprocessing import LabelBinarizer
from .metrics.pairwise import euclidean_distances
from .utils import check_random_state
from .utils.validation import _num_samples
from .utils.validation import check_is_fitted
from .utils.validation import check_X_y, check_array
from .utils.multiclass import (_check_partial_fit_first_call,
                               check_classification_targets,
                               _ovr_decision_function)
from .utils.metaestimators import _safe_split, if_delegate_has_method

from .externals.joblib import Parallel
from .externals.joblib import delayed
from .externals.six.moves import zip as izip

class OneVsRestClassifier(BaseEstimator, ClassifierMixin, MetaEstimatorMixin):
    """One-vs-the-rest (OvR) multiclass/multilabel strategy
    Also known as one-vs-all, this strategy consists in fitting one classifier
    per class. For each classifier, the class is fitted against all the other
    classes. In addition to its computational efficiency (only `n_classes`
    classifiers are needed), one advantage of this approach is its
    interpretability. Since each class is represented by one and one classifier
    only, it is possible to gain knowledge about the class by inspecting its
    corresponding classifier. This is the most commonly used strategy for
    multiclass classification and is a fair default choice.
    This strategy can also be used for multilabel learning, where a classifier
    is used to predict multiple labels for instance, by fitting on a 2-d matrix
    in which cell [i, j] is 1 if sample i has label j and 0 otherwise.
    In the multilabel learning literature, OvR is also known as the binary
    relevance method.
    Read more in the :ref:`User Guide <ovr_classification>`.
    Parameters
    ----------
    estimator : estimator object
        An estimator object implementing `fit` and one of `decision_function`
        or `predict_proba`.
    n_jobs : int, optional, default: 1
        The number of jobs to use for the computation. If -1 all CPUs are used.
        If 1 is given, no parallel computing code is used at all, which is
        useful for debugging. For n_jobs below -1, (n_cpus + 1 + n_jobs) are
        used. Thus for n_jobs = -2, all CPUs but one are used.
    Attributes
    ----------
    estimators_ : list of `n_classes` estimators
        Estimators used for predictions.
    classes_ : array, shape = [`n_classes`]
        Class labels.
    label_binarizer_ : LabelBinarizer object
        Object used to transform multiclass labels to binary labels and
        vice-versa.
    multilabel_ : boolean
        Whether a OneVsRestClassifier is a multilabel classifier.
    """
    def __init__(self, estimator, n_jobs=1):
        self.estimator = estimator
        self.n_jobs = n_jobs

    def fit(self, X, y,emb_sampler, lmd=lmd, **DW_PARAMS):
        """Fit underlying estimators.
        Parameters
        ----------
        X : (sparse) array-like, shape = [n_samples, n_features]
            Data.
        y : (sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]
            Multi-class targets. An indicator matrix turns on multilabel
            classification.
        Returns
        -------
        self
        """
        # A sparse LabelBinarizer, with sparse_output=True, has been shown to
        # outpreform or match a dense label binarizer in all cases and has also
        # resulted in less or equal memory consumption in the fit_ovr function
        # overall.
        self.label_binarizer_ = LabelBinarizer(sparse_output=True)
        Y = self.label_binarizer_.fit_transform(y)
        Y = Y.tocsc()
        self.classes_ = self.label_binarizer_.classes_
        columns = (col.toarray().ravel() for col in Y.T)
        # In cases where individual estimators are very fast to train setting
        # n_jobs > 1 in can results in slower performance due to the overhead
        # of spawning threads.  See joblib issue #112.
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(delayed(_fit_binary)(
            self.estimator, X, column, classes=[
                "not %s" % self.label_binarizer_.classes_[i],
                self.label_binarizer_.classes_[i]])
            for i, column in enumerate(columns))

        return self

    @if_delegate_has_method('estimator')
    def partial_fit(self, X, y, classes=None):
        """Partially fit underlying estimators
        Should be used when memory is inefficient to train all data.
        Chunks of data can be passed in several iteration.
        Parameters
        ----------
        X : (sparse) array-like, shape = [n_samples, n_features]
            Data.
        y : (sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]
            Multi-class targets. An indicator matrix turns on multilabel
            classification.
        classes : array, shape (n_classes, )
            Classes across all calls to partial_fit.
            Can be obtained via `np.unique(y_all)`, where y_all is the
            target vector of the entire dataset.
            This argument is only required in the first call of partial_fit
            and can be omitted in the subsequent calls.
        Returns
        -------
        self
        """
        if _check_partial_fit_first_call(self, classes):
            if not hasattr(self.estimator, "partial_fit"):
                raise ValueError(("Base estimator {0}, doesn't have "
                                 "partial_fit method").format(self.estimator))
            self.estimators_ = [clone(self.estimator) for _ in range
                                (self.n_classes_)]

            # A sparse LabelBinarizer, with sparse_output=True, has been
            # shown to outperform or match a dense label binarizer in all
            # cases and has also resulted in less or equal memory consumption
            # in the fit_ovr function overall.
            self.label_binarizer_ = LabelBinarizer(sparse_output=True)
            self.label_binarizer_.fit(self.classes_)

        if len(np.setdiff1d(y, self.classes_)):
            raise ValueError(("Mini-batch contains {0} while classes " +
                             "must be subset of {1}").format(np.unique(y),
                                                             self.classes_))

        Y = self.label_binarizer_.transform(y)
        Y = Y.tocsc()
        columns = (col.toarray().ravel() for col in Y.T)

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_partial_fit_binary)(estimator, X, column)
            for estimator, column in izip(self.estimators_, columns))

        return self

    def predict(self, X):
        """Predict multi-class targets using underlying estimators.
        Parameters
        ----------
        X : (sparse) array-like, shape = [n_samples, n_features]
            Data.
        Returns
        -------
        y : (sparse) array-like, shape = [n_samples, ], [n_samples, n_classes].
            Predicted multi-class targets.
        """
        check_is_fitted(self, 'estimators_')
        if (hasattr(self.estimators_[0], "decision_function") and
                is_classifier(self.estimators_[0])):
            thresh = 0
        else:
            thresh = .5

        n_samples = _num_samples(X)
        if self.label_binarizer_.y_type_ == "multiclass":
            maxima = np.empty(n_samples, dtype=float)
            maxima.fill(-np.inf)
            argmaxima = np.zeros(n_samples, dtype=int)
            for i, e in enumerate(self.estimators_):
                pred = _predict_binary(e, X)
                np.maximum(maxima, pred, out=maxima)
                argmaxima[maxima == pred] = i
            return self.classes_[np.array(argmaxima.T)]
        else:
            indices = array.array('i')
            indptr = array.array('i', [0])
            for e in self.estimators_:
                indices.extend(np.where(_predict_binary(e, X) > thresh)[0])
                indptr.append(len(indices))
            data = np.ones(len(indices), dtype=int)
            indicator = sp.csc_matrix((data, indices, indptr),
                                      shape=(n_samples, len(self.estimators_)))
            return self.label_binarizer_.inverse_transform(indicator)

    @if_delegate_has_method(['_first_estimator', 'estimator'])
    def predict_proba(self, X):
        """Probability estimates.
        The returned estimates for all classes are ordered by label of classes.
        Note that in the multilabel case, each sample can have any number of
        labels. This returns the marginal probability that the given sample has
        the label in question. For example, it is entirely consistent that two
        labels both have a 90% probability of applying to a given sample.
        In the single label multiclass case, the rows of the returned matrix
        sum to 1.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
        Returns
        -------
        T : (sparse) array-like, shape = [n_samples, n_classes]
            Returns the probability of the sample for each class in the model,
            where classes are ordered as they are in `self.classes_`.
        """
        check_is_fitted(self, 'estimators_')
        # Y[i, j] gives the probability that sample i has the label j.
        # In the multi-label case, these are not disjoint.
        Y = np.array([e.predict_proba(X)[:, 1] for e in self.estimators_]).T

        if len(self.estimators_) == 1:
            # Only one estimator, but we still want to return probabilities
            # for two classes.
            Y = np.concatenate(((1 - Y), Y), axis=1)

        if not self.multilabel_:
            # Then, probabilities should be normalized to 1.
            Y /= np.sum(Y, axis=1)[:, np.newaxis]
        return Y

    @if_delegate_has_method(['_first_estimator', 'estimator'])
    def decision_function(self, X):
        """Returns the distance of each sample from the decision boundary for
        each class. This can only be used with estimators which implement the
        decision_function method.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
        Returns
        -------
        T : array-like, shape = [n_samples, n_classes]
        """
        check_is_fitted(self, 'estimators_')
        if len(self.estimators_) == 1:
            return self.estimators_[0].decision_function(X)
        return np.array([est.decision_function(X).ravel()
                         for est in self.estimators_]).T

    @property
    def multilabel_(self):
        """Whether this is a multilabel classifier"""
        return self.label_binarizer_.y_type_.startswith('multilabel')

    @property
    def n_classes_(self):
        return len(self.classes_)

    @property
    def coef_(self):
        check_is_fitted(self, 'estimators_')
        if not hasattr(self.estimators_[0], "coef_"):
            raise AttributeError(
                "Base estimator doesn't have a coef_ attribute.")
        coefs = [e.coef_ for e in self.estimators_]
        if sp.issparse(coefs[0]):
            return sp.vstack(coefs)
        return np.vstack(coefs)

    @property
    def intercept_(self):
        check_is_fitted(self, 'estimators_')
        if not hasattr(self.estimators_[0], "intercept_"):
            raise AttributeError(
                "Base estimator doesn't have an intercept_ attribute.")
        return np.array([e.intercept_.ravel() for e in self.estimators_])

    @property
    def _pairwise(self):
        """Indicate if wrapped estimator is using a precomputed Gram matrix"""
        return getattr(self.estimator, "_pairwise", False)

    @property
    def _first_estimator(self):
        return self.estimators_[0]

ImportError: No module named BaseEstimator

In [78]:
NUM_READS = 1000
NUM_WEAK_CLASSIFIERS = 30
TREE_DEPTH = 4
DW_PARAMS = {'num_reads': NUM_READS,
            'auto_scale': True,
            'num_spin_reversal_transforms': 10,
            'postprocess': 'optimization',
            'dwave_sampler' : DWaveSampler(token=sapi_token, endpoint = url),
            'emb_sampler' : EmbeddingComposite(dwave_sampler)
            }
set_params(**params)
# define sampler
dwave_sampler = DWaveSampler(token=sapi_token, endpoint = url)
emb_sampler = EmbeddingComposite(dwave_sampler)

lmd = 1.0

classifier = OneVsRestClassifier(QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH))
y_score = classifier.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
y_pred = classifier.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print y_pred, acc


ValueError: Invalid parameter num_spin_reversal_transforms for estimator OneVsRestClassifier(estimator=<qboost.qboost.QBoostClassifier object at 0x113fdefd0>,
          n_jobs=1). Check the list of available parameters with `estimator.get_params().keys()`.